In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential  # Fix: Import Sequential
from tensorflow.keras.layers import LSTM, Dense
import urllib.request
from sklearn.model_selection import train_test_split

# Download text data
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
urllib.request.urlretrieve(url, 'shakespeare.txt')
text = open('shakespeare.txt', 'r').read()

# Create character mappings
chars = sorted(list(set(text)))
char_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_char = {i:ch for i, ch in enumerate(chars)}

# Prepare sequences
max_length = 100
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - max_length, step):
    sentences.append(text[i:i + max_length])
    next_chars.append(text[i + max_length])

# Vectorization
X = np.zeros((len(sentences), max_length, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_to_int[char]] = 1
    y[i, char_to_int[next_chars[i]]] = 1

# Build model
model = Sequential()
model.add(LSTM(128, input_shape=(max_length, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Train model
history = model.fit(X, y, batch_size=128, epochs=30, validation_split=0.1)

# Generate text
def generate_text(seed, length=400):
    generated = seed
    for _ in range(length):
        x_pred = np.zeros((1, max_length, len(chars)))
        for t, char in enumerate(seed):
            x_pred[0, t, char_to_int[char]] = 1

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = np.argmax(preds)
        next_char = int_to_char[next_index]

        generated += next_char
        seed = seed[1:] + next_char
    return generated

# Calculate perplexity
def calculate_perplexity(model, X, y):
    y_pred = model.predict(X)
    cross_entropy = tf.keras.losses.categorical_crossentropy(y, y_pred)
    perplexity = np.exp(np.mean(cross_entropy))
    return perplexity

perplexity = calculate_perplexity(model, X, y)
print(f"Perplexity: {perplexity}")

# Generate sample text
print(generate_text("ROMEO: "))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
2614/2614 ━━━━━━━━━━━━━━━━━━━━ 26s 9ms/step - loss: 2.7553 - val_loss: 2.1565
Epoch 2/30
2614/2614 ━━━━━━━━━━━━━━━━━━━━ 24s 9ms/step - loss: 2.0669 - val_loss: 2.0055
Epoch 3/30
2614/2614 ━━━━━━━━━━━━━━━━━━━━ 42s 10ms/step - loss: 1.8982 - val_loss: 1.9242
Epoch 4/30
2614/2614 ━━━━━━━━━━━━━━━━━━━━ 24s 9ms/step - loss: 1.7972 - val_loss: 1.8814
Epoch 5/30
2614/2614 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - loss: 1.7232 - val_loss: 1.8344
Epoch 6/30
2614/2614 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - loss: 1.6716 - val_loss: 1.8029
Epoch 7/30
2614/2614 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - loss: 1.6339 - val_loss: 1.7790
Epoch 8/30
2614/2614 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - loss: 1.6030 - val_loss: 1.7617
Epoch 9/30
2614/2614 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - loss: 1.5685 - val_loss: 1.7486
Epoch 10/30
2614/2614 ━━━━━━━━━━━━━━━━━━━━ 24s 9ms/step - loss: 1.5463 - val_loss: 1.7396
Epoch 11/30
2614/2614 ━━━━━━━━━━━━━━━━━━━━ 42s 10ms/step - loss: 1.5262 - val_loss: 1.7204
Epoch 12/30
2614/